# Dynamic Trade Generation (Inside Simulation Loop)

This notebook demonstrates how to generate external trades **dynamically during backtesting** based on current portfolio state.

## Why Dynamic Generation?

Dynamic trade generation allows your strategy to:
- React to portfolio performance and volatility
- Adjust positions based on current holdings
- Implement conditional trading logic (e.g., rebalance only on month-end)
- Use historical returns and PnL to inform decisions
- Model realistic trading behavior

This is more realistic than pre-generating all trades, as it mimics how traders actually make decisions.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from backtesting import (
    Backtester, BacktestConfig, DataManager,
    create_simple_signal_generator,
    TargetWeightSignalGenerator,
    AlphaSignalGenerator,
    ConditionalSignalGenerator,
    TradeGeneratorConfig,
    generate_external_trades_from_signals
)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## Setup

In [ ]:
# Initialize data manager and backtester
data_manager = DataManager('../data')

config = BacktestConfig(
    initial_cash=1000000,
    transaction_cost_bps=10,
    max_portfolio_variance=0.015
)

backtester = Backtester(data_manager, config)

print("Backtester initialized")

## Example 1: Simple Callable Function

The simplest way - pass a function that generates trades based on current state.

In [ ]:
def generate_daily_trades(context):
    """
    Generate trades based on portfolio performance.
    
    Strategy: Increase exposure when portfolio is up, reduce when down.
    """
    # Need at least 10 days of history
    if len(context['daily_returns']) < 10:
        return {}
    
    # Calculate recent performance
    recent_return = sum(context['daily_returns'][-10:])
    
    # Adjust target weights based on performance
    if recent_return > 0.05:  # Up 5%
        target_weights = {
            'AAPL': 0.35,
            'MSFT': 0.30,
            'GOOGL': 0.25
        }
    elif recent_return < -0.05:  # Down 5%
        target_weights = {
            'AAPL': 0.15,
            'MSFT': 0.15,
            'GOOGL': 0.10
        }
    else:
        target_weights = {
            'AAPL': 0.25,
            'MSFT': 0.20,
            'GOOGL': 0.15
        }
    
    # Convert to trades
    trades = generate_external_trades_from_signals(
        signals=target_weights,
        current_positions=context['portfolio'].positions,
        close_prices=context['prices'],
        portfolio_value=context['portfolio_value'],
        signal_type='weights'
    )
    
    return trades

# Run backtest with dynamic generation
print("Running backtest with simple callable...")
results = backtester.run(
    start_date='2023-01-01',
    end_date='2023-06-30',
    use_case=3,
    inputs={'external_trades': generate_daily_trades}  # Pass function!
)

# Results
metrics = results.calculate_metrics()
print("\n=== Performance Metrics ===")
print(f"Total Return: {metrics['total_return']:.2%}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Max Drawdown: {metrics['max_drawdown']:.2%}")

## Example 2: Using create_simple_signal_generator

Convenient wrapper for quick setup with signal history tracking.

In [ ]:
def my_signals(context):
    """Calculate target weights based on volatility."""
    if len(context['daily_returns']) < 20:
        return {}
    
    # Calculate recent volatility
    recent_vol = np.std(context['daily_returns'][-20:]) * np.sqrt(252)
    
    # Adjust exposure based on volatility
    if recent_vol > 0.25:  # High vol - reduce exposure
        base_weight = 0.15
    elif recent_vol < 0.15:  # Low vol - increase exposure
        base_weight = 0.30
    else:
        base_weight = 0.20
    
    return {
        'AAPL': base_weight,
        'MSFT': base_weight * 0.9,
        'GOOGL': base_weight * 0.8,
        'AMZN': base_weight * 0.7
    }

# Create signal generator
signal_gen = create_simple_signal_generator(
    signal_function=my_signals,
    signal_type='weights'
)

print("Running backtest with signal generator...")
results = backtester.run(
    start_date='2023-01-01',
    end_date='2023-06-30',
    use_case=3,
    inputs={'external_trades': signal_gen}
)

# Results
metrics = results.calculate_metrics()
print("\n=== Performance Metrics ===")
print(f"Total Return: {metrics['total_return']:.2%}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")

# View signal history
signal_history = signal_gen.get_history()
print(f"\nGenerated {len(signal_history)} signals across {signal_history['date'].nunique()} days")
print("\nSample signals:")
print(signal_history.head(10))

## Example 3: Target Weight Signal Generator

More control with custom configuration.

In [ ]:
def rebalancing_weights(context):
    """
    Rebalance when weights drift too much from target.
    """
    # Target allocation
    target_allocation = {
        'AAPL': 0.25,
        'MSFT': 0.25,
        'GOOGL': 0.20,
        'AMZN': 0.15,
        'TSLA': 0.15
    }
    
    # Calculate current weights
    portfolio_value = context['portfolio_value']
    current_positions = context['portfolio'].positions
    prices = context['prices']
    
    current_weights = {}
    for ticker, shares in current_positions.items():
        if ticker in prices:
            value = shares * prices[ticker]
            current_weights[ticker] = value / portfolio_value
    
    # Check if rebalancing is needed (5% drift threshold)
    needs_rebalance = False
    for ticker, target_weight in target_allocation.items():
        current_weight = current_weights.get(ticker, 0)
        if abs(current_weight - target_weight) > 0.05:
            needs_rebalance = True
            break
    
    return target_allocation if needs_rebalance else {}

# Configure trade generation with multiple fills
trade_config = TradeGeneratorConfig(
    price_impact_bps=5.0,
    num_fills_per_ticker=3,  # Simulate VWAP
    max_adv_participation=0.10
)

signal_gen = TargetWeightSignalGenerator(
    signal_function=rebalancing_weights,
    trade_generator_config=trade_config
)

print("Running backtest with target weight generator...")
results = backtester.run(
    start_date='2023-01-01',
    end_date='2023-06-30',
    use_case=3,
    inputs={'external_trades': signal_gen}
)

metrics = results.calculate_metrics()
print("\n=== Performance Metrics ===")
print(f"Total Return: {metrics['total_return']:.2%}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")

# Analyze trades
trades_summary = results.get_external_trades_summary()
print("\n=== External Trades Summary ===")
print(trades_summary)

## Example 4: Conditional Signal Generator

Only trade when specific conditions are met.

In [ ]:
def is_month_end(context):
    """Check if today is month end."""
    date = context['date']
    return date.is_month_end

def monthly_weights(context):
    """Target weights for monthly rebalancing."""
    return {
        'AAPL': 0.30,
        'MSFT': 0.25,
        'GOOGL': 0.20,
        'AMZN': 0.15,
        'TSLA': 0.10
    }

signal_gen = ConditionalSignalGenerator(
    signal_function=monthly_weights,
    condition_function=is_month_end,
    signal_type='weights'
)

print("Running backtest with conditional generator (month-end only)...")
results = backtester.run(
    start_date='2023-01-01',
    end_date='2023-06-30',
    use_case=3,
    inputs={'external_trades': signal_gen}
)

metrics = results.calculate_metrics()
print("\n=== Performance Metrics ===")
print(f"Total Return: {metrics['total_return']:.2%}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")

# Check trading days
trades_by_date = results.get_external_trades_by_date()
print(f"\nTraded on {len(trades_by_date)} days (should be ~6 month-ends)")
print("\nTrading dates:")
print(trades_by_date[['date', 'num_trades', 'num_tickers', 'total_notional']])

## Example 5: Complex Strategy with State Awareness

A more sophisticated strategy that adjusts based on multiple factors.

In [ ]:
def adaptive_strategy(context):
    """
    Adaptive strategy that adjusts weights based on:
    - Portfolio performance
    - Volatility
    - Drawdown
    """
    # Need sufficient history
    if len(context['daily_returns']) < 30:
        return {}
    
    # Calculate metrics
    recent_returns = context['daily_returns'][-30:]
    cumulative_return = np.prod([1 + r for r in recent_returns]) - 1
    volatility = np.std(recent_returns) * np.sqrt(252)
    
    # Calculate drawdown
    portfolio_values = context['portfolio_value']
    # Simplified drawdown calculation
    
    # Base weights
    base_weights = {
        'AAPL': 0.25,
        'MSFT': 0.20,
        'GOOGL': 0.20,
        'AMZN': 0.15,
        'TSLA': 0.10
    }
    
    # Adjust leverage based on volatility
    if volatility > 0.30:
        leverage = 0.6  # Reduce in high vol
    elif volatility < 0.15:
        leverage = 1.1  # Increase in low vol
    else:
        leverage = 0.9
    
    # Adjust based on performance
    if cumulative_return < -0.15:  # Down 15%
        leverage *= 0.7  # Cut exposure
    elif cumulative_return > 0.20:  # Up 20%
        leverage *= 0.85  # Take some profits
    
    # Scale weights
    target_weights = {k: v * leverage for k, v in base_weights.items()}
    
    # Check if adjustment is significant
    current_positions = context['portfolio'].positions
    prices = context['prices']
    pv = context['portfolio_value']
    
    current_weights = {}
    for ticker, shares in current_positions.items():
        if ticker in prices:
            current_weights[ticker] = (shares * prices[ticker]) / pv
    
    # Only rebalance if drift > 3%
    needs_rebalance = any(
        abs(current_weights.get(t, 0) - w) > 0.03
        for t, w in target_weights.items()
    )
    
    return target_weights if needs_rebalance else {}

# Use with advanced configuration
trade_config = TradeGeneratorConfig(
    price_impact_bps=7.0,
    num_fills_per_ticker=5,
    max_adv_participation=0.08
)

signal_gen = TargetWeightSignalGenerator(
    signal_function=adaptive_strategy,
    trade_generator_config=trade_config
)

print("Running backtest with adaptive strategy...")
results = backtester.run(
    start_date='2023-01-01',
    end_date='2023-12-31',
    use_case=3,
    inputs={'external_trades': signal_gen}
)

# Comprehensive analysis
metrics = results.calculate_metrics()
print("\n=== Performance Metrics ===")
print(f"Total Return: {metrics['total_return']:.2%}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Max Drawdown: {metrics['max_drawdown']:.2%}")
print(f"Calmar Ratio: {metrics.get('calmar_ratio', 0):.2f}")

# PnL breakdown
pnl_df = results.get_pnl_breakdown_dataframe()
print("\n=== PnL Breakdown Summary ===")
print(f"Total External PnL: ${pnl_df['external_pnl'].sum():,.0f}")
print(f"Total Executed PnL: ${pnl_df['executed_pnl'].sum():,.0f}")
print(f"Total Overnight PnL: ${pnl_df['overnight_pnl'].sum():,.0f}")

# Trading activity
trades_by_date = results.get_external_trades_by_date()
print(f"\nTraded on {len(trades_by_date)} days out of ~252 trading days")
print(f"Average trades per day: {trades_by_date['num_trades'].mean():.1f}")
print(f"Total transaction costs: ${trades_by_date['total_cost'].sum():,.0f}")

# Signal history
signal_history = signal_gen.get_history()
print(f"\nSignal history: {len(signal_history)} signals generated")

## Example 6: Comparing Static vs Dynamic

Compare pre-generated trades vs dynamic generation.

In [ ]:
from backtesting import ExternalTradeGenerator

# Static: Pre-generate all trades
print("Strategy 1: Static pre-generated trades")
print("="*50)

prices_df = data_manager.get_prices()
dates = prices_df.index[:100]  # First 100 days

# Generate all trades upfront
generator = ExternalTradeGenerator()
target_positions_by_date = {}

for date in dates:
    # Simple fixed allocation
    target_positions_by_date[date] = {
        'AAPL': 100,
        'MSFT': 100,
        'GOOGL': 50
    }

static_trades = generator.generate_multi_day_trades(
    dates=dates,
    target_positions_by_date=target_positions_by_date,
    prices_df=prices_df,
    initial_positions={}
)

results_static = backtester.run(
    start_date=dates[0],
    end_date=dates[-1],
    use_case=3,
    inputs={'external_trades': static_trades}
)

metrics_static = results_static.calculate_metrics()
print(f"Return: {metrics_static['total_return']:.2%}")
print(f"Sharpe: {metrics_static['sharpe_ratio']:.2f}")
print(f"Max DD: {metrics_static['max_drawdown']:.2%}")

# Dynamic: Generate during backtest
print("\nStrategy 2: Dynamic generation (adjusts to performance)")
print("="*50)

def dynamic_positions(context):
    """Adjust positions based on recent performance."""
    if len(context['daily_returns']) < 5:
        base_size = 100
    else:
        recent_return = sum(context['daily_returns'][-5:])
        
        if recent_return > 0.02:  # Up 2%
            base_size = 120  # Increase
        elif recent_return < -0.02:  # Down 2%
            base_size = 80  # Decrease
        else:
            base_size = 100
    
    return {
        'AAPL': base_size,
        'MSFT': base_size,
        'GOOGL': base_size // 2
    }

signal_gen_dynamic = create_simple_signal_generator(
    signal_function=dynamic_positions,
    signal_type='positions'
)

results_dynamic = backtester.run(
    start_date=dates[0],
    end_date=dates[-1],
    use_case=3,
    inputs={'external_trades': signal_gen_dynamic}
)

metrics_dynamic = results_dynamic.calculate_metrics()
print(f"Return: {metrics_dynamic['total_return']:.2%}")
print(f"Sharpe: {metrics_dynamic['sharpe_ratio']:.2f}")
print(f"Max DD: {metrics_dynamic['max_drawdown']:.2%}")

# Comparison
print("\nComparison:")
print("="*50)
print(f"Dynamic outperformance: {(metrics_dynamic['total_return'] - metrics_static['total_return']) * 100:.2f}%")
print(f"\nDynamic strategy adjusted positions based on performance,")
print(f"while static used fixed positions throughout.")

## Summary

This notebook demonstrated:

1. **Simple callable functions** - Pass a function directly to the backtester
2. **Signal generator classes** - Use pre-built generators for common patterns
3. **Conditional trading** - Only trade when specific conditions are met
4. **State-aware strategies** - Adjust based on portfolio performance and volatility
5. **Static vs Dynamic** - Compare pre-generated vs dynamic trade generation

### Key Advantages of Dynamic Generation

- **Realistic**: Mimics how traders actually make decisions
- **Adaptive**: React to portfolio state and market conditions
- **Flexible**: Easy to implement complex conditional logic
- **Memory efficient**: No need to pre-generate all trades
- **Trackable**: Signal history available for analysis

### Next Steps

- Implement your own signal generation logic
- Test different rebalancing conditions
- Combine with risk management (Use Case 3 optimization)
- Analyze execution quality and PnL attribution
- Generate comprehensive reports with `results.generate_full_report()`